In [1]:
import numpy as np

In [2]:
MAX_MODE = 'MAX'
MIN_MODE = 'MIN'

In [3]:
class SimplexMethod:
    def __init__(self, c, a, b, mode):
        self.main_variables_count = a.shape[1]
        self.restrictions_count = a.shape[0]
        self.variables_count = self.main_variables_count + \
            self.restrictions_count  
        self.mode = mode 

        # coefficients of the function
        self.c = np.concatenate([c, np.zeros((self.restrictions_count + 1))])
        self.f = np.zeros((self.variables_count + 1))
        self.basis = [
            i + self.main_variables_count for i in range(self.restrictions_count)]

        self.init_table(a, b)

    # initializing the table
    def init_table(self, a, b):
        self.table = np.zeros(
            (self.restrictions_count, self.variables_count + 1))
        for i in range(self.restrictions_count):
            for j in range(self.main_variables_count):
                self.table[i][j] = a[i][j]

            for j in range(self.restrictions_count):
                self.table[i][j + self.main_variables_count] = int(i == j)
                self.table[i][-1] = b[i]

    def get_negative_b_row(self):
        row = -1
        for i, a_row in enumerate(self.table):
            if a_row[-1] < 0 and (row == -1 or abs(a_row[-1]) > abs(self.table[row][-1])):
                row = i
        return row

    def get_negative_b_column(self, row):
        column = -1

        for i, aij in enumerate(self.table[row][:-1]):
            if aij < 0 and (column == -1 or abs(aij) > abs(self.table[row][column])):
                column = i

        return column

    # removing negative free coefficients
    def remove_negative_b(self):
        while True:
            row = self.get_negative_b_row()

            if row == -1:
                return True 

            column = self.get_negative_b_column(row)

            if column == -1:
                return False  # failed
            self.gauss(row, column) 
            self.calculate_f()
            print('\nLeaving variable has been removed in row:', row + 1)
            self.print_table()

    def gauss(self, row, column):
        self.table[row] /= self.table[row][column]
        for i in range(self.restrictions_count):
            if i != row:
                self.table[i] -= self.table[row] * self.table[i][column]
        self.basis[row] = column

    # F values
    def calculate_f(self):
        for i in range(self.variables_count + 1):
            self.f[i] = -self.c[i]

            for j in range(self.restrictions_count):
                self.f[i] += self.c[self.basis[j]] * self.table[j][i]

    # calculation of simplex relations
    def get_relations(self, column):
        q = []

        for i in range(self.restrictions_count):
            if self.table[i][column] == 0:
                q.append(np.inf)

            else:
                q_i = self.table[i][-1] / \
                    self.table[i][column]  # ratio
                q.append(q_i if q_i >= 0 else np.inf)

        return q

    # solution
    def get_solve(self):
        y = np.zeros((self.variables_count))

        for i in range(self.restrictions_count):
            y[self.basis[i]] = self.table[i][-1]

        return y

    # decision
    def solve(self):
        print('\nIteration 0')
        self.calculate_f()
        self.print_table()

        if not self.remove_negative_b():
            print('Solve does not exist')
            return False

        iteration = 1

        while True:
            self.calculate_f()
            print('\nIteration', iteration)
            self.print_table()

            # optimal
            if all(fi >= 0 if self.mode == MAX_MODE else fi <= 0 for fi in self.f[:-1]):
                break

            column = (np.argmin if self.mode == MAX_MODE else np.argmax)(
                self.f[:-1])
            q = self.get_relations(column)

            if all(qi == np.inf for qi in q): 
                print('Solve does not exist')
                return False

            self.gauss(np.argmin(q), column)
            iteration += 1

        return True  # solution

    # simplex table output
    def print_table(self):
        print('     |' + ''.join(['   y%-3d |' % (i + 1)
              for i in range(self.variables_count)]) + '    b   |')

        for i in range(self.restrictions_count):
            print('%4s |' % ('y' + str(self.basis[i] + 1)) + ''.join(
                [' %6.2f |' % aij for j, aij in enumerate(self.table[i])]))

        print('   F |' + ''.join([' %6.2f |' % aij for aij in self.f]))
        print('   y |' + ''.join([' %6.2f |' % xi for xi in self.get_solve()]))

    # coefficient output (y1,y2,y3,...)
    def print_coef(self, ai, i):
        if ai == 1:
            return 'y%d' % (i + 1)

        if ai == -1:
            return '-y%d' % (i + 1)

        return '%.2fy%d' % (ai, i + 1)

    # output task
    def print_task(self, full=False):
        print(' + '.join(['%.2fy%d' % (ci, i + 1) for i, ci in enumerate(
            self.c[:self.main_variables_count]) if ci != 0]), '-> ', self.mode)

        for row in self.table:
            if full:
                print(' + '.join([self.print_coef(ai, i) for i, ai in enumerate(
                    row[:self.variables_count]) if ai != 0]), '=', row[-1])
            else:
                print(' + '.join([self.print_coef(ai, i) for i, ai in enumerate(
                    row[:self.main_variables_count]) if ai != 0]), '<=', row[-1])

In [4]:
# translation into a dual task

def make_dual(a, b, c):
    return -a.T, -c, b

c = np.array([-2, -3, -5, -6])  # RHS
a = np.array([
    [-1, -2, -3, -1],
    [-2, 1, -1, 3]
])

b = np.array([2, 3])  # objective fn

a, b, c = make_dual(a, b, c)  # turned into a dual task
simplex = SimplexMethod(c, a, b, MIN_MODE)  # turn into max mode

print("Dual: ")
simplex.print_task()
simplex.solve()

Dual: 
2.00y1 + 3.00y2 ->  MIN
y1 + 2.00y2 <= 2.0
2.00y1 + -y2 <= 3.0
3.00y1 + y2 <= 5.0
y1 + -3.00y2 <= 6.0

Iteration 0
     |   y1   |   y2   |   y3   |   y4   |   y5   |   y6   |    b   |
  y3 |   1.00 |   2.00 |   1.00 |   0.00 |   0.00 |   0.00 |   2.00 |
  y4 |   2.00 |  -1.00 |   0.00 |   1.00 |   0.00 |   0.00 |   3.00 |
  y5 |   3.00 |   1.00 |   0.00 |   0.00 |   1.00 |   0.00 |   5.00 |
  y6 |   1.00 |  -3.00 |   0.00 |   0.00 |   0.00 |   1.00 |   6.00 |
   F |  -2.00 |  -3.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |
   y |   0.00 |   0.00 |   2.00 |   3.00 |   5.00 |   6.00 |

Iteration 1
     |   y1   |   y2   |   y3   |   y4   |   y5   |   y6   |    b   |
  y3 |   1.00 |   2.00 |   1.00 |   0.00 |   0.00 |   0.00 |   2.00 |
  y4 |   2.00 |  -1.00 |   0.00 |   1.00 |   0.00 |   0.00 |   3.00 |
  y5 |   3.00 |   1.00 |   0.00 |   0.00 |   1.00 |   0.00 |   5.00 |
  y6 |   1.00 |  -3.00 |   0.00 |   0.00 |   0.00 |   1.00 |   6.00 |
   F |  -2.00 |  -3.00 |   0.00 | 

True